In [1]:
import sys
import os
ref_path = r"D:\OneDrive\\References\Farahani_ALS-main\codes"  ## https://github.com/netneurolab/Farahani_ALS/tree/main/codes
sys.path.append(ref_path)

import warnings
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from neuromaps.images import load_data
from neuromaps.images import dlabel_to_gifti
from netneurotools.datasets import fetch_schaefer2018
from functions import parcel2fsLR, save_gifti
from globals import path_results, path_sc, path_fig, nnodes, path_networks,path_networks_ALS
from functions import save_parcellated_data_in_Schaefer_forVis
from scipy.io import loadmat, savemat
#References: https://github.com/netneurolab/Farahani_ALS/tree/2ac249a22d1973f4758a9d6c0968c1d9cc73bebf/codes

In [2]:
#------------------------------------------------------------------------------
# Needed functions
#------------------------------------------------------------------------------

def get_neighbour_deformation(node_deformation, sc, fc=None, gce =None,rs=None):
    '''
    compute mean neighbour deformation of each node

    Inputs:

    node_deformation = n x 1 matrix of node deformations
    sc = n x n structural connectivity matrix (binary or weighted)
    fc = n x n functional connectivity matrix (optional)

    Output:

        neighbour_deformation = n x 1 matrix of mean neighbour deformations
    '''

    nnodes = len(node_deformation)
    neighbour_deformation = np.zeros((nnodes,))

    FC = np.ones((nnodes, nnodes))
    GCE = np.ones((nnodes, nnodes))
    RS = np.ones((nnodes, nnodes))


    if fc is not None:
        FC = fc
    if gce is not None:
        GCE = gce
    if rs is not None:
        RS = rs

    for i in range(nnodes):
        neighbour_deformation[i] = np.mean(node_deformation[sc[i, :] != 0]
                                           * FC[sc[i, :] != 0, i]
                                           * GCE[sc[i, :] != 0, i]
                                           * RS[sc[i, :] != 0, i])

    return neighbour_deformation

def get_epicentre_likelihood(node_deformation, neighbour_deformation,
                             spins=None, nspins=None):
    
    ## reference: https://github.com/netneurolab/hansen_crossdisorder_vulnerability/blob/54d34055bd765a79777437f362fbaf338403926e/code/02_network_spreading.py#L49
    
    '''
    epicenter likelihood ordered same as node and neighbour deformation

    node_deformation: (n, ) array_like
        Node deformation ordered such that a small value represents more atrophy.
    neighbour_deformation: (n, ) array_like
        Neighbour deformation ordered such that a small value represents more atrophy.

    Returns
    -------
    epicentre_likelihood: (n, ) ndarray
        Epicentre likelihood, where a larger value represents greater likelihood.
    epicentre_pvals: (n, ) ndarray
        Spin-test one-tailed p-value of epicentre likelihood.
    
    '''
    a = np.argsort(node_deformation)[::-1]
    b = np.zeros((len(a), ))
    for k in range(len(a)):
        b[a[k]] = k
    aa = np.argsort(neighbour_deformation)[::-1]
    bb = np.zeros((len(aa), ))
    for k in range(len(aa)):
        bb[aa[k]] = k

    epicentre_likelihood = np.mean(np.stack((b, bb), axis=1), axis=1)
    epicentre_pvals = np.ones(epicentre_likelihood.shape)

    if spins is not None:
        null = np.zeros((len(node_deformation), nspins))
        for k in range(nspins):
            a = np.argsort(node_deformation[spins[:, k]])[::-1]
            b = np.zeros((len(a), ))
            for j in range(len(a)):
                b[a[j]] = j
            null[:, k] = np.mean(np.stack((b, bb), axis=1), axis=1)
        epicentre_pvals = (1 + np.sum(null > np.expand_dims(epicentre_likelihood, 1),axis=1)) / (nspins + 1)

    return epicentre_likelihood, epicentre_pvals

In [3]:
# Load disease atropy map (W-score map)

tmp  = loadmat('D:\\OneDrive\\5_TMS_response\\CT_Wscore53avg_S400_7Net_vHCP2.mat')
W_score_avg = tmp['Wscore_s400_7Net']
disease_profile = np.reshape(W_score_avg, 400)

In [4]:
#Load networks
## https://github.com/netneurolab/hansen_many_networks/tree/085151024a2f8dda1f07ce13e946def1aec93f17/data/Schaefer400

hansen_dir ='D:\\OneDrive\\GraduateStudent_Phd1\\5_SubstanceUse_topography\\References\\hansen_many_networks-master\\data\\Schaefer400\\'
sc = np.load(hansen_dir + 'consensusSC.npy')
fc = np.load(hansen_dir + 'haemodynamic_connectivity.npy')

gc = np.load(hansen_dir + 'gene_coexpression.npy')
rs = np.load(hansen_dir + 'receptor_similarity.npy')

In [5]:
#spin test
from netneurotools import datasets, freesurfer, stats

annot = datasets.fetch_schaefer2018('fsaverage5')
annot = annot['400Parcels7Networks']
coords, hemi = freesurfer.find_parcel_centroids(lhannot=annot.lh,
                                                rhannot=annot.rh,
                                                version='fsaverage5',
                                                surf='sphere',
                                                method='surface')
spins = stats.gen_spinsamples(coords, hemi, method='vasa', seed=2345, n_rotate=1000)


In [6]:
from scipy.io import savemat
version ='4modes_dataHansenSSC'

epi=get_epicentre_likelihood(disease_profile,get_neighbour_deformation(disease_profile,sc,fc=fc, gce =gc,rs=rs),spins=spins,nspins=1000)
epi1 = epi[0]
savemat(f"D:\\OneDrive\\5_TMS_response\\Epicenter\\SC\\epicenterMap_avgranking_S4007Net_{version}.mat", {"my_array": epi1}) 


In [7]:
## check significance of the epicenters
from neuromaps import images, nulls
from netneurotools import datasets as nntdata
nspins=1000
rotated_epiMap_TMS = nulls.alexander_bloch(epi1, atlas='fsLR', density='32k', parcellation=dlabel_to_gifti( nntdata.fetch_schaefer2018('fslr32k')['400Parcels7Networks']),
                                n_perm=1000, seed=2345)

epi1 = np.asarray(epi1).reshape(-1, 1)
p_values = (1 + np.sum(rotated_epiMap_TMS > epi1, axis=1)) / (nspins + 1)
savemat(f"D:\\OneDrive\\5_TMS_response\\Epicenter\\SC\\epicenterMap_avgranking_S4007Net_{version}_pvals.mat", {"my_array": p_values}) 


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
